In [1]:
import dgl
import copy
import glob
import math
import torch
import torch.nn as nn
import numpy as np
from os import path
from pathlib import Path
from trainAndEval import train, evaluate
from dgl.data import DGLDataset
from TrainResults import TrainResults
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from DGL_GCN import Deep_DGL_GCN
from ToyDGLDataset_v2 import ToyDGLDataset_v2, GetNodeFeatureVectors, GetEdgeFeatureVectors, GetNeighborNodes, GetEdgeList

Using backend: pytorch


In [2]:
# training
def trainEpochs(model, device, dataloader, optimizer, loss_fn, batchsize, nEpochs):
    results = TrainResults()
    results.startTrainingTimer()
    bestModel = None
    bestValAcc = 0.0

    for epoch in range(nEpochs):
        # train
        epochLoss = train(model, device, dataloader, optimizer, loss_fn, batchsize, results)

        # evaluate
        train_result = evaluate(model, device, train_dataloader, loss_fn)
        val_result = evaluate(model, device, val_dataloader, loss_fn)
        test_result = evaluate(model, device, test_dataloader, loss_fn)

        results.addEpochResult(epochLoss, train_result, val_result, test_result)

        #if(epoch % 5 == 0):
        #    results.printLastResult()

        if results.best_val_acc > bestValAcc:
            bestValAcc = results.best_val_acc
            bestModel = copy.deepcopy(model)
            
    results.endTrainingTimer()

    return results, bestModel

In [3]:
def getAllDatasetNames(datasetRootDir):
    files = glob.glob(datasetRootDir + '/*/*/*.json', recursive=True)
    #files = [x for x in files if "Toy2_v3_0" in x]
    files.sort()
    datasetDirectories = [path.dirname(file) for file in files]
    datasetnames = [path.normpath(dir).split(path.sep)[-1] for dir in datasetDirectories]
    return datasetDirectories, datasetnames

In [4]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

datasetRootDir = '/home/andrew/GNN_Sandbox/GraphToyDatasets_v3'
datasetDirs, datasetNames = getAllDatasetNames(datasetRootDir)

In [5]:
import time
now = time.time()

batchSize = 1024
print(f'Device: {device}')

for nLayer in range(2,11):
    print(f'##############################################################')
    print(f'----------------LAYER COUNT: {nLayer}-------------------')
    for i in range(len(datasetDirs)):
        dataset = ToyDGLDataset_v2(datasetNames[i], datasetDirs[i])
        splitIndices = dataset.get_split_indices()

        train_sampler = SubsetRandomSampler(splitIndices['train'])
        val_sampler = SubsetRandomSampler(splitIndices['valid'])
        test_sampler = SubsetRandomSampler(splitIndices['test'])

        train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
        val_dataloader = GraphDataLoader(dataset, sampler=val_sampler, batch_size=batchSize, drop_last=False)
        test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)

        # Create the model with given dimensions
        model = Deep_DGL_GCN(dataset.dim_nfeats, 16, dataset.num_graph_classes, nLayer, 0).to(device)
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
        loss_fn = nn.CrossEntropyLoss()
        epochs = 30

        # train
        print(f'Beginning training on dataset {datasetNames[i]}')
        results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
        results.printBestResult()

        # save results
        outputFolder = path.join(path.join(datasetDirs[i], "ThesisPlots/Deepness"), f'DGL_GCN_Layers_{nLayer}')
        Path(outputFolder).mkdir(parents=True, exist_ok=True)

        results.savePlots(outputFolder)
        results.dumpSummary(outputFolder)
        results.pickledump(outputFolder)

        # save the best model for inference. (when loading for inference -> model.eval()!! )
        # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
        torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))

        print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')

Device: cpu
##############################################################
----------------LAYER COUNT: 2-------------------
Done loading data from cached files.
Beginning training on dataset Toy0_v3_1_0NodeCountMeanDiff
Best epoch: 
Epoch: 14, Loss: 0.6951, Validation Loss: 0.6932, Train: 0.498, Validation: 0.504, Test: 0.492, AUC: 0.493

The training took 243 seconds (4.05 minutes).
------------------(1/27) models trained------------------

Done loading data from cached files.
Beginning training on dataset Toy0_v3_1_11NodeCountMeanDiff
Best epoch: 
Epoch: 0, Loss: 1.0954, Validation Loss: 0.5822, Train: 0.498, Validation: 0.502, Test: 0.506, AUC: 0.740

The training took 237 seconds (3.95 minutes).
------------------(2/27) models trained------------------

Done loading data from cached files.
Beginning training on dataset Toy0_v3_1_1NodeCountMeanDiff
Best epoch: 
Epoch: 0, Loss: 0.8353, Validation Loss: 0.6932, Train: 0.498, Validation: 0.502, Test: 0.506, AUC: 0.500

The training to

KeyboardInterrupt: 